__LLAMA BOT ALPHA PREVIEW__ \
__by:__ Guillermo Javier Auza Banegas\
__Motivation:__ Skill Evaluation \
__Company:__ LlamitAI \
__Date:__ Feb 2025

For this notebook to work, ollama must be serving an LLM model and an embedding model, in this case deepseek and nomic respectively \
\
First of all, libraries must be imported, in this case langchain and its variants: community, llm, etc.

In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

Now, we define the functions that will be used during our code:

In [17]:
def load_documents(path="./docs/txt/"): # default path is a double folder with txt files
    loader = DirectoryLoader(path, glob="*.txt") # read all txt files in the path
    docs = loader.load()
    return docs

def chunk_splitter(docs): # argument previously split documents
    text_splitter = RecursiveCharacterTextSplitter( 
        chunk_size = 750, 
        chunk_overlap = 300,
        length_function = len,
        add_start_index = True,
        )
    chunks = text_splitter.split_documents(docs) # turn each document into chunks using parameters from text splitter
    return chunks

def format_docs(docs): # turn documents into a single string
    return "\n\n".join(doc.page_content for doc in docs)

After declaring the functions, we can start defining the models and getting the embeddings from the text:

In [18]:
documents = load_documents() # load documents
chunks = chunk_splitter(documents) # turn list of docs into list of chunks

embedding_model = OllamaEmbeddings(model="nomic-embed-text") # load a model for embedding text
vectorstore = Chroma.from_documents(documents = chunks, embedding = embedding_model) # store embeddings in a vector store

model = OllamaLLM( # Define our LLM model and its parameters in this case, deepseek r1 with 14b params
    model="deepseek-r1:14b",
    temperature = 0,
    max_tokens = None,
    timeout = None,
    max_retries = 1,
    )

Now, lets define a template for our propts and for this example, a predefined question or petition

In [19]:
template = """Responde a la peticion cubierto en llaves de manera simple y concisa.
Para esta tarea utiliza el contexto en sus llaves correspondientes.

<contexto>
contexto : {context}
</contexto>

<peticion>
peticion: {question}
</peticion>
"""

question = """¿Quiénes son los demandados?
¿Quiénes son los demandantes?
¿Quién es el juez encargado del caso?
¿Que tipo de proceso es este?
¿Cuando y donde sucedio la demanda?"""

Finally, lets define the chain of operations for the model so we can invoke a simplified a chain of operations for the model

In [20]:
docs = vectorstore.similarity_search(question) # Search for similiraty in the embeddings from the question and our vector store
context = format_docs(docs) # Setup the document chunk as a string for context

prompt = ChatPromptTemplate.from_template(template) # create the prompt from our template with pending context and petition
chain = prompt | model # establish the chain of operation from the propt to the model

response = chain.invoke({"context": context, "question": question}) # invoke the chain of operations with determined context and petition

Finally, lets see what the model respond to our question based on the following document:\
\
15-ENE 2025 9:14 \
TDJ \
TRIBU NAD DEPARTAMENTALL \
DE \
S \
JUSTICLA \
LAPAZ \
A \
112 \
ASFI \
15 ENE 2025 \
10791 \
ESTADO PLURINACIONAL DE BOLIVIA \
"LP" \
CIUDAD DE LA PAZ \
NUREJ: 204177105 \
AUTORIDAD DE SUPERVISION\
DEL SISTEMA FINANCIERO\
EXP: 196/24\
No. TRÁMITE\
Cite Of. 99/2024\
E\
T-1211991226\
E\
Lugar y fecha: La Paz, 25 de octubre de 2024\
Señores: AUTORIDAD DE SUPERVISIÓN DEL SISTEMA FINANCIERO - ASFI La Paz\
REF.: RETENCIÓN DE FONDOS A: 1. RAFAEL PEREZ BLANCO CON C.I. 8765402-LP.\
DEMANDANTE: 1. BANCO DEL PUEBLO S.A.\
PROCESO: CIVIL EJECUTIVO\
MONTO A RETENER: EN CAJA DE AHORRO, CUENTA CORRIENTE, DEPÓSITOS\
A PLAZO FIJO, TÍTULOS, VALORES Y OTROS SIMILARES, HASTA EL MONTO\
ADEUDADO DE Bs.- 15.010.00.- (SON QUINCE MIL DIEZ 00/100 BOLIVIANOS)\
Así se tiene ORDENADO mediante Sentencia inicial Nro. 756/2024 de fecha 20 de\
septiembre de 2024 a fs. 76 a 78 de obrados, siempre y cuando no se trate del pago\
de salarios, de ser así estese a las previsiones del Art. 48 parágrafo IV de la\
Constitución Política del Estado.\
FCM/CCC\
DEPARTAMENTAL\
JUZGADO\
PÚBLICO CIVILY\
\
COMERCIAL 3°\
M Sc DAEN Fausto Calle M.\
JUEZ PUBLICO CIVIL y COMERICIAL 3\
DEPARTAMENTO BE JUSTICIA\
SELLO DE JUZGADO\
FIRMADEOIUBZ\
Paz-Bolivia\
SELLO

In [24]:
from IPython.display import display, Markdown
display(Markdown(response))

<think>
Bueno, tengo que responder a las preguntas de la petición utilizando el contexto proporcionado. Primero, identificaré cada pregunta y buscaré la información correspondiente en el contexto.

1. **Demandados**: En el contexto se menciona "RAFAEL PEREZ BLANCO" como el demandado con su CI.
2. **Demandantes**: El_demandante es "BANCO DEL PUEBLO S.A." según el texto.
3. **Juez encargado**: El juez es MSc DAEN Fausto Calle M., del Juzgado Público Civil y Comercial 3°.
4. **Tipo de proceso**: Es un proceso "CIVIL EJECUTIVO".
5. **Cuándo y dónde sucedió la demanda**: La sentencia fue el 20 de septiembre de 2024, en el Juzgado Público Civil y Comercial 3°.

Ahora, estructuraré las respuestas de manera clara y concisa.
</think>

**Respuesta a la petición:**

1. **Demandados**: Rafael Pérez Blanco, con CI 8765402-LP.
2. **Demandantes**: Banco del Pueblo S.A.
3. **Juez encargado**: MSc DAEN Fausto Calle M., Juzgado Público Civil y Comercial 3°.
4. **Tipo de proceso**: Proceso civil ejecutivo.
5. **Cuándo y dónde sucedió la demanda**: El proceso se tramitó en el Juzgado Público Civil y Comercial 3°, con una sentencia emitida el 20 de septiembre de 2024.